In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import torch.nn.functional as F

In [2]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

In [4]:
# CIFAR-100 dataset
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

In [5]:
# Load pretrained ResNet-50
model = models.resnet50(pretrained=True)

/home/BTECH_7TH_SEM/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/BTECH_7TH_SEM/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/BTECH_7TH_SEM/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████████████████████████████████| 97.8M/97.8M [00:02<00:00, 49.5MB/s]


In [6]:
# Modify the final layer and add one extra FC layer
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 100)
)

model = model.to(device)

In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)

In [8]:
# Training
for epoch in range(25):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/25], Loss: {running_loss/len(trainloader):.4f}')

Epoch [1/25], Loss: 2.4918
Epoch [2/25], Loss: 1.6187
Epoch [3/25], Loss: 1.3441
Epoch [4/25], Loss: 1.1926
Epoch [5/25], Loss: 1.0790
Epoch [6/25], Loss: 0.9929
Epoch [7/25], Loss: 0.9191
Epoch [8/25], Loss: 0.8558
Epoch [9/25], Loss: 0.8165
Epoch [10/25], Loss: 0.7671
Epoch [11/25], Loss: 0.7243
Epoch [12/25], Loss: 0.6966
Epoch [13/25], Loss: 0.6566
Epoch [14/25], Loss: 0.6301
Epoch [15/25], Loss: 0.5933
Epoch [16/25], Loss: 0.5736
Epoch [17/25], Loss: 0.5631
Epoch [18/25], Loss: 0.5304
Epoch [19/25], Loss: 0.5104
Epoch [20/25], Loss: 0.4950
Epoch [21/25], Loss: 0.4731
Epoch [22/25], Loss: 0.4592
Epoch [23/25], Loss: 0.4496
Epoch [24/25], Loss: 0.4291
Epoch [25/25], Loss: 0.4079


In [9]:
# Testing
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Test Accuracy: 73.40%
